In [14]:
#cp_major = major.partial_molar_cp
#wdot_major = major.net_production_rates
#print(cp_major)
#print(wdot_major) 

import cantera as ct
import numpy as np

#use reaction mechanism GRI-Mech 3.0
methane = ct.Solution('gri30.xml')
ethane = ct.Solution('gri30.xml')
propane = ct.Solution('gri30.xml')
#define stechiometric fuel and gas mixtures
methane.TPX = 1200, ct.one_atm, 'CH4:1, O2:2, N2:7.52'
ethane.TPX = 1200, ct.one_atm, 'C2H6:1, O2:3.5, N2:13.6'
propane.TPX = 1200, ct.one_atm, 'C3H8:1, O2:5, N2:18.8'
#define reactors
combustor = ct.Reactor(methane)
igniter = ct.Reactor(methane)
crankcase = ct.Reservoir

print(methane.species_names)

['H2', 'H', 'O', 'O2', 'OH', 'H2O', 'HO2', 'H2O2', 'C', 'CH', 'CH2', 'CH2(S)', 'CH3', 'CH4', 'CO', 'CO2', 'HCO', 'CH2O', 'CH2OH', 'CH3O', 'CH3OH', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'C2H5', 'C2H6', 'HCCO', 'CH2CO', 'HCCOH', 'N', 'NH', 'NH2', 'NH3', 'NNH', 'NO', 'NO2', 'N2O', 'HNO', 'CN', 'HCN', 'H2CN', 'HCNN', 'HCNO', 'HOCN', 'HNCO', 'NCO', 'N2', 'AR', 'C3H7', 'C3H8', 'CH2CHO', 'CH3CHO']
